# Problem Statement

Investing in Flats is the biggest investment for most households in cities like Singapore which is very
small but heavily urbanised. Therefore, being able to accurately value the flat prices will not only
facilitate market transactions by providing valuable guidance for all market participants (be it home
owners, home buyers, landlords, tenants or banks that underwrite mortgages), but also provide
useful insights for policy makers and government authorities in understanding the current state of
the economy.

# Importing the libraries

In [186]:
import numpy as np
import pandas as pd
from pandas import DataFrame 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
# !pip install seaborn
import seaborn as sns
#!pip install imblearn
#if the above command does not work to install imblearn package run the following command in your terminal
# conda install -c glemaitre imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Ignore the warnings

In [188]:
import warnings
warnings.filterwarnings('ignore')

# Reading the input data files

In [189]:
Flat_data_withoutremaining_lease = pd.read_csv("train1.csv", sep=',', header=0, na_values='unknown')
Flat_data_1 =  pd.read_csv("train2.csv", sep=',', header=0, na_values='unknown')
Flat_data_2  = pd.read_csv("train3.csv", sep=',', header=0, na_values='unknown')
Bulding_to_mrt_data= pd.read_csv("building_to_mrt_distance.csv", sep=',', header=0, na_values='unknown')

 # Chekcing the shapes

In [191]:
Flat_data_withoutremaining_lease.shape

(52203, 11)

In [192]:
Flat_data_1.head()

month        town flat_type block        street_name storey_range  \
0  2015-01  ANG MO KIO    3 ROOM   174   ANG MO KIO AVE 4     07 TO 09   
1  2015-01  ANG MO KIO    3 ROOM   541  ANG MO KIO AVE 10     01 TO 03   
2  2015-01  ANG MO KIO    3 ROOM   163   ANG MO KIO AVE 4     01 TO 03   
3  2015-01  ANG MO KIO    3 ROOM   446  ANG MO KIO AVE 10     01 TO 03   
4  2015-01  ANG MO KIO    3 ROOM   557  ANG MO KIO AVE 10     07 TO 09   

   floor_area_sqm      flat_model  lease_commence_date  remaining_lease  \
0            60.0        Improved                 1986               70   
1            68.0  New Generation                 1981               65   
2            69.0  New Generation                 1980               64   
3            68.0  New Generation                 1979               63   
4            68.0  New Generation                 1980               64   

   resale_price  uniqueID  
0      255000.0    152203  
1      275000.0    152204  
2      285000.0    152205  
3      290000.0    152206  
4      290000.0    152207

In [193]:
Flat_data_1.shape

(37153, 12)

In [194]:
Flat_data_2.shape

(32578, 12)

# Concatenate the files Train 2 and Train 3

In [197]:
Flat_data = pd.concat([Flat_data_1,Flat_data_2], ignore_index=True) 


In [198]:
Flat_data.shape

(69731, 12)

# Append with Train 1

In [199]:
Flat_data=Flat_data.append(Flat_data_withoutremaining_lease,ignore_index=True )

In [200]:
Flat_data.shape

(121934, 12)

In [201]:
Flat_data.head(2)

block      flat_model flat_type  floor_area_sqm  lease_commence_date  \
0   174        Improved    3 ROOM            60.0                 1986   
1   541  New Generation    3 ROOM            68.0                 1981   

     month remaining_lease  resale_price storey_range        street_name  \
0  2015-01              70      255000.0     07 TO 09   ANG MO KIO AVE 4   
1  2015-01              65      275000.0     01 TO 03  ANG MO KIO AVE 10   

         town  uniqueID  
0  ANG MO KIO    152203  
1  ANG MO KIO    152204

# Check NA volumes

In [202]:
Flat_data.isnull().sum()

block                      0
flat_model                 0
flat_type                  0
floor_area_sqm             0
lease_commence_date        0
month                      0
remaining_lease        52203
resale_price               0
storey_range               0
street_name                0
town                       0
uniqueID                   0
dtype: int64

# Check the data types

In [203]:
Flat_data.dtypes

block                   object
flat_model              object
flat_type               object
floor_area_sqm         float64
lease_commence_date      int64
month                   object
remaining_lease         object
resale_price           float64
storey_range            object
street_name             object
town                    object
uniqueID                 int64
dtype: object

# Splitting the values in column Month

In [204]:
Flat_data['Year'] = Flat_data['month'].str.split('-').str[0]

In [205]:
Flat_data.drop("month", axis = 1, inplace= True)

In [206]:
Flat_data.head(2)

block      flat_model flat_type  floor_area_sqm  lease_commence_date  \
0   174        Improved    3 ROOM            60.0                 1986   
1   541  New Generation    3 ROOM            68.0                 1981   

  remaining_lease  resale_price storey_range        street_name        town  \
0              70      255000.0     07 TO 09   ANG MO KIO AVE 4  ANG MO KIO   
1              65      275000.0     01 TO 03  ANG MO KIO AVE 10  ANG MO KIO   

   uniqueID  Year  
0    152203  2015  
1    152204  2015

# Coverting the data types

In [215]:
for col in ['lease_commence_date', 'remaining_lease', 'resale_price', 'Year']:
    Flat_data[col] = Flat_data[col].astype('float64')

In [216]:
for col in ['floor_area_sqm','flat_model', 'flat_type', 'storey_range', 'street_name', 'town']:
    Flat_data[col] = Flat_data[col].astype('category')

# Getting the remaining lease column missing values from Year and Lease commence date columns


In [214]:
Flat_data["remaining_lease"]= Flat_data["lease_commence_date"].add(99)

In [220]:
Flat_data["remaining_lease"]= Flat_data["remaining_lease"].subtract(Flat_data["Year"]) 

# Create a new column "Building" from street name and Block

In [221]:
for col in ['street_name','block']:
    Flat_data[col] = Flat_data[col].astype('str')

In [222]:
Flat_data["Building"]= Flat_data["street_name"].map(str) +' '+'BLK'+' '+Flat_data["block"].map(str)

In [223]:
Flat_data=Flat_data.drop("block", axis=1)

In [224]:
Flat_data=Flat_data.drop("street_name", axis=1)

In [225]:
Flat_data['Building'] = Flat_data['Building'].astype('category')

In [226]:
Flat_data.head()

flat_model flat_type floor_area_sqm  lease_commence_date  \
0        Improved    3 ROOM           60.0               1986.0   
1  New Generation    3 ROOM           68.0               1981.0   
2  New Generation    3 ROOM           69.0               1980.0   
3  New Generation    3 ROOM           68.0               1979.0   
4  New Generation    3 ROOM           68.0               1980.0   

   remaining_lease  resale_price storey_range        town  uniqueID    Year  \
0             70.0      255000.0     07 TO 09  ANG MO KIO    152203  2015.0   
1             65.0      275000.0     01 TO 03  ANG MO KIO    152204  2015.0   
2             64.0      285000.0     01 TO 03  ANG MO KIO    152205  2015.0   
3             63.0      290000.0     01 TO 03  ANG MO KIO    152206  2015.0   
4             64.0      290000.0     07 TO 09  ANG MO KIO    152207  2015.0   

                    Building  
0   ANG MO KIO AVE 4 BLK 174  
1  ANG MO KIO AVE 10 BLK 541  
2   ANG MO KIO AVE 4 BLK 163  
3  ANG MO KIO AVE 10 BLK 446  
4  ANG MO KIO AVE 10 BLK 557

# Merging the flat data and Bulding_to_mrt_data

In [233]:
Building_merge_data = pd.merge(Flat_data,Bulding_to_mrt_data,on='Building',how='left')

In [234]:
Building_merge_data1 = pd.merge(Flat_data,Bulding_to_mrt_data,on='Building',how='left')

In [235]:
Building_merge_data.shape

(121934, 130)

In [236]:
Building_merge_data.head(2)

flat_model flat_type floor_area_sqm  lease_commence_date  \
0        Improved    3 ROOM           60.0               1986.0   
1  New Generation    3 ROOM           68.0               1981.0   

   remaining_lease  resale_price storey_range        town  uniqueID    Year  \
0             70.0      255000.0     07 TO 09  ANG MO KIO    152203  2015.0   
1             65.0      275000.0     01 TO 03  ANG MO KIO    152204  2015.0   

   ... Bendemeer MRT station  Geylang Bahru MRT Station  Mattar MRT station  \
0  ...           7421.041681                7081.755148         7352.239556   
1  ...           6680.085927                6016.771386         5926.544362   

   Ubi MRT station  Kaki Bukit MRT Station  Bedok North MRT station  \
0      8596.977026             9133.510791             10044.626630   
1      6870.053507             7267.824892              8122.857984   

   Bedok Reservoir MRT station  Tampines West MRT station  \
0                 11422.489390               11677.874430   
1                  9436.891823                9634.790047   

   Tampines East MRT station  Upper Changi MRT station  
0                13199.56790               14274.52606  
1                11118.75881               12219.54327  

[2 rows x 130 columns]

In [237]:
Building_merge_data.head()

flat_model flat_type floor_area_sqm  lease_commence_date  \
0        Improved    3 ROOM           60.0               1986.0   
1  New Generation    3 ROOM           68.0               1981.0   
2  New Generation    3 ROOM           69.0               1980.0   
3  New Generation    3 ROOM           68.0               1979.0   
4  New Generation    3 ROOM           68.0               1980.0   

   remaining_lease  resale_price storey_range        town  uniqueID    Year  \
0             70.0      255000.0     07 TO 09  ANG MO KIO    152203  2015.0   
1             65.0      275000.0     01 TO 03  ANG MO KIO    152204  2015.0   
2             64.0      285000.0     01 TO 03  ANG MO KIO    152205  2015.0   
3             63.0      290000.0     01 TO 03  ANG MO KIO    152206  2015.0   
4             64.0      290000.0     07 TO 09  ANG MO KIO    152207  2015.0   

   ... Bendemeer MRT station  Geylang Bahru MRT Station  Mattar MRT station  \
0  ...           7421.041681                7081.755148         7352.239556   
1  ...           6680.085927                6016.771386         5926.544362   
2  ...           7216.672100                6880.699707         7162.224183   
3  ...           6089.113498                5467.845178         5464.360263   
4  ...           6475.019803                5786.810519         5673.489182   

   Ubi MRT station  Kaki Bukit MRT Station  Bedok North MRT station  \
0      8596.977026             9133.510791             10044.626630   
1      6870.053507             7267.824892              8122.857984   
2      8424.258160             8974.449251              9892.559060   
3      6532.350261             7012.780876              7910.451995   
4      6608.865123             7010.544853              7869.935438   

   Bedok Reservoir MRT station  Tampines West MRT station  \
0                 11422.489390               11677.874430   
1                  9436.891823                9634.790047   
2                 11281.057630               11551.775750   
3                  9283.019841                9554.228026   
4                  9192.151102                9403.044071   

   Tampines East MRT station  Upper Changi MRT station  
0                13199.56790               14274.52606  
1                11118.75881               12219.54327  
2                13094.58415               14152.34412  
3                11130.67541               12158.06694  
4                10908.40733               11992.39127  

[5 rows x 130 columns]

# Feature Engineering - removing the unwanted index column

In [241]:
Building_merge_data=Building_merge_data.drop("uniqueID", axis=1)

In [242]:
Building_merge_data.head()

flat_model flat_type floor_area_sqm  lease_commence_date  \
0        Improved    3 ROOM           60.0               1986.0   
1  New Generation    3 ROOM           68.0               1981.0   
2  New Generation    3 ROOM           69.0               1980.0   
3  New Generation    3 ROOM           68.0               1979.0   
4  New Generation    3 ROOM           68.0               1980.0   

   remaining_lease  resale_price storey_range        town    Year  \
0             70.0      255000.0     07 TO 09  ANG MO KIO  2015.0   
1             65.0      275000.0     01 TO 03  ANG MO KIO  2015.0   
2             64.0      285000.0     01 TO 03  ANG MO KIO  2015.0   
3             63.0      290000.0     01 TO 03  ANG MO KIO  2015.0   
4             64.0      290000.0     07 TO 09  ANG MO KIO  2015.0   

                    Building  ...  Bendemeer MRT station  \
0   ANG MO KIO AVE 4 BLK 174  ...            7421.041681   
1  ANG MO KIO AVE 10 BLK 541  ...            6680.085927   
2   ANG MO KIO AVE 4 BLK 163  ...            7216.672100   
3  ANG MO KIO AVE 10 BLK 446  ...            6089.113498   
4  ANG MO KIO AVE 10 BLK 557  ...            6475.019803   

   Geylang Bahru MRT Station  Mattar MRT station  Ubi MRT station  \
0                7081.755148         7352.239556      8596.977026   
1                6016.771386         5926.544362      6870.053507   
2                6880.699707         7162.224183      8424.258160   
3                5467.845178         5464.360263      6532.350261   
4                5786.810519         5673.489182      6608.865123   

   Kaki Bukit MRT Station  Bedok North MRT station  \
0             9133.510791             10044.626630   
1             7267.824892              8122.857984   
2             8974.449251              9892.559060   
3             7012.780876              7910.451995   
4             7010.544853              7869.935438   

   Bedok Reservoir MRT station  Tampines West MRT station  \
0                 11422.489390               11677.874430   
1                  9436.891823                9634.790047   
2                 11281.057630               11551.775750   
3                  9283.019841                9554.228026   
4                  9192.151102                9403.044071   

   Tampines East MRT station  Upper Changi MRT station  
0                13199.56790               14274.52606  
1                11118.75881               12219.54327  
2                13094.58415               14152.34412  
3                11130.67541               12158.06694  
4                10908.40733               11992.39127  

[5 rows x 129 columns]

# Exploratory Data Analysis

Resale Price data is normally distributed

Resale prices has plunged in 2015 and slowly recovered

Not much correlation between the independent variables in flat data

# Splitting the categorical and numerical columns

In [252]:
Building_merge_data_cat_resale=Building_merge_data

In [253]:
Building_merge_data.head()

flat_model flat_type floor_area_sqm  lease_commence_date  \
0        Improved    3 ROOM           60.0               1986.0   
1  New Generation    3 ROOM           68.0               1981.0   
2  New Generation    3 ROOM           69.0               1980.0   
3  New Generation    3 ROOM           68.0               1979.0   
4  New Generation    3 ROOM           68.0               1980.0   

   remaining_lease  resale_price storey_range        town    Year  \
0             70.0      255000.0     07 TO 09  ANG MO KIO  2015.0   
1             65.0      275000.0     01 TO 03  ANG MO KIO  2015.0   
2             64.0      285000.0     01 TO 03  ANG MO KIO  2015.0   
3             63.0      290000.0     01 TO 03  ANG MO KIO  2015.0   
4             64.0      290000.0     07 TO 09  ANG MO KIO  2015.0   

                    Building  ...  Bendemeer MRT station  \
0   ANG MO KIO AVE 4 BLK 174  ...            7421.041681   
1  ANG MO KIO AVE 10 BLK 541  ...            6680.085927   
2   ANG MO KIO AVE 4 BLK 163  ...            7216.672100   
3  ANG MO KIO AVE 10 BLK 446  ...            6089.113498   
4  ANG MO KIO AVE 10 BLK 557  ...            6475.019803   

   Geylang Bahru MRT Station  Mattar MRT station  Ubi MRT station  \
0                7081.755148         7352.239556      8596.977026   
1                6016.771386         5926.544362      6870.053507   
2                6880.699707         7162.224183      8424.258160   
3                5467.845178         5464.360263      6532.350261   
4                5786.810519         5673.489182      6608.865123   

   Kaki Bukit MRT Station  Bedok North MRT station  \
0             9133.510791             10044.626630   
1             7267.824892              8122.857984   
2             8974.449251              9892.559060   
3             7012.780876              7910.451995   
4             7010.544853              7869.935438   

   Bedok Reservoir MRT station  Tampines West MRT station  \
0                 11422.489390               11677.874430   
1                  9436.891823                9634.790047   
2                 11281.057630               11551.775750   
3                  9283.019841                9554.228026   
4                  9192.151102                9403.044071   

   Tampines East MRT station  Upper Changi MRT station  
0                13199.56790               14274.52606  
1                11118.75881               12219.54327  
2                13094.58415               14152.34412  
3                11130.67541               12158.06694  
4                10908.40733               11992.39127  

[5 rows x 129 columns]

In [254]:
#Building_merge_data.dtypes

In [255]:
Building_merge_data_cat= Building_merge_data.select_dtypes(include=['category'])
Building_merge_data_num= Building_merge_data.select_dtypes(include=['float64'])

In [256]:
Building_merge_data_cat_cols= list(Building_merge_data.select_dtypes(include=['category']).columns)
Building_merge_data_num_cols= Building_merge_data.select_dtypes(include=['float64']).columns

In [257]:
Building_merge_data_cat.head()

flat_model flat_type floor_area_sqm storey_range        town
0        Improved    3 ROOM           60.0     07 TO 09  ANG MO KIO
1  New Generation    3 ROOM           68.0     01 TO 03  ANG MO KIO
2  New Generation    3 ROOM           69.0     01 TO 03  ANG MO KIO
3  New Generation    3 ROOM           68.0     01 TO 03  ANG MO KIO
4  New Generation    3 ROOM           68.0     07 TO 09  ANG MO KIO

# Scaling the Data

In [262]:
scaler = StandardScaler()

scaler.fit(Building_merge_data_num)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [263]:
Building_merge_data_num_std=pd.DataFrame(scaler.transform(Building_merge_data_num), columns=Building_merge_data_num_cols)



In [264]:
Building_merge_data_num_std.head()

lease_commence_date  remaining_lease  resale_price      Year  \
0            -0.445791        -0.457388     -1.432988  0.048286   
1            -0.901839        -0.916395     -1.285628  0.048286   
2            -0.993049        -1.008196     -1.211948  0.048286   
3            -1.084259        -1.099998     -1.175108  0.048286   
4            -0.993049        -1.008196     -1.175108  0.048286   

   Jurong East MRT station  Bukit Batok MRT station  Bukit Gombak MRT station  \
0                -0.253066                -0.335911                 -0.387212   
1                 0.034128                -0.021876                 -0.056240   
2                -0.253741                -0.332572                 -0.380233   
3                -0.010187                -0.057518                 -0.083570   
4                 0.051501                 0.000264                 -0.030391   

   Choa Chu Kang MRT station  Yew Tee MRT station  Kranji MRT station  ...  \
0                  -0.371122            -0.414448           -0.527826  ...   
1                  -0.042630            -0.080400           -0.202074  ...   
2                  -0.355595            -0.394875           -0.498582  ...   
3                  -0.045510            -0.072372           -0.163896  ...   
4                  -0.010724            -0.044989           -0.158867  ...   

   Bendemeer MRT station  Geylang Bahru MRT Station  Mattar MRT station  \
0              -0.646379                  -0.623356           -0.527765   
1              -0.807390                  -0.842040           -0.796591   
2              -0.690789                  -0.664641           -0.563594   
3              -0.935810                  -0.954757           -0.883740   
4              -0.851952                  -0.889261           -0.844307   

   Ubi MRT station  Kaki Bukit MRT Station  Bedok North MRT station  \
0        -0.352137               -0.287176                -0.215187   
1        -0.638513               -0.577431                -0.496682   
2        -0.380779               -0.311922                -0.237462   
3        -0.694514               -0.617109                -0.527794   
4        -0.681826               -0.617457                -0.533729   

   Bedok Reservoir MRT station  Tampines West MRT station  \
0                    -0.136833                  -0.125597   
1                    -0.406322                  -0.395431   
2                    -0.156028                  -0.142251   
3                    -0.427205                  -0.406071   
4                    -0.439538                  -0.426038   

   Tampines East MRT station  Upper Changi MRT station  
0                  -0.089115                 -0.064169  
1                  -0.350263                 -0.320821  
2                  -0.102291                 -0.079429  
3                  -0.348768                 -0.328498  
4                  -0.376663                 -0.349190  

[5 rows x 123 columns]

In [265]:
scaler = StandardScaler()

scaler.fit(test_Building_merge_data_num)

StandardScaler(copy=True, with_mean=True, with_std=True)

# Onehot Encoding of categorical data

In [268]:
ohe=OneHotEncoder(handle_unknown='ignore',sparse=False)

In [269]:
Building_merge_data_cat=pd.DataFrame(ohe.fit_transform(Building_merge_data_cat))

In [270]:
Building_merge_data_cat.head(2)




0    1    2    3    4    5    6    7    8    9    ...  239  240  241  242  \
0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

   243  244  245  246  247  248  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  

[2 rows x 249 columns]

In [274]:
pip install prince

Note: you may need to restart the kernel to use updated packages.


In [275]:
import prince
from sklearn.decomposition import PCA


pcanum = PCA(n_components=10).fit(Building_merge_data_num_std)

In [276]:
pcadata = pd.DataFrame(pcanum.transform(Building_merge_data_num_std), columns=['pc1','pc2','pc3','pc4','pc5','pc6','p7','pc8','pc9','pc10'])
                                                                                         
                                                                                         
                                                                                         

In [277]:
pcadata.head()

pc1       pc2       pc3       pc4       pc5       pc6        p7  \
0 -3.960537 -3.762787 -4.693950 -1.168886 -0.648010 -0.223176 -0.931093   
1 -5.835641 -2.250867 -4.255435 -1.639094 -0.747809 -0.369905 -0.656918   
2 -4.310199 -3.990938 -4.601152 -1.707895 -0.548483 -0.254743 -0.562933   
3 -6.587737 -3.068126 -4.139848 -1.766792 -0.754117 -0.415867 -0.464232   
4 -6.186999 -2.287985 -4.136039 -1.685416 -0.746922 -0.400777 -0.531645   

        pc8       pc9      pc10  
0 -0.103455  0.096092  0.638992  
1 -0.005389  0.505846  0.325636  
2  0.227478  0.289917  0.569773  
3  0.014396  0.556946  0.426652  
4  0.036848  0.547659  0.289866

In [278]:
pcanum.explained_variance_ratio_

array([0.54169512, 0.32828675, 0.07952375, 0.01726963, 0.00867352,
       0.00823359, 0.00466091, 0.00439369, 0.00307207, 0.00117104])

In [279]:
Flat_building_pp=pd.concat([pcadata, Building_merge_data_cat, Building_merge_data1['resale_price']], axis = 1)

In [343]:
Flat_building_pp.head()

pc1       pc2       pc3       pc4       pc5       pc6        p7  \
0 -3.960537 -3.762787 -4.693950 -1.168886 -0.648010 -0.223176 -0.931093   
1 -5.835641 -2.250867 -4.255435 -1.639094 -0.747809 -0.369905 -0.656918   
2 -4.310199 -3.990938 -4.601152 -1.707895 -0.548483 -0.254743 -0.562933   
3 -6.587737 -3.068126 -4.139848 -1.766792 -0.754117 -0.415867 -0.464232   
4 -6.186999 -2.287985 -4.136039 -1.685416 -0.746922 -0.400777 -0.531645   

        pc8       pc9      pc10  ...  240  241  242  243  244  245  246  247  \
0 -0.103455  0.096092  0.638992  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1 -0.005389  0.505846  0.325636  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.227478  0.289917  0.569773  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3  0.014396  0.556946  0.426652  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4  0.036848  0.547659  0.289866  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   248  resale_price  
0  0.0      255000.0  
1  0.0      275000.0  
2  0.0      285000.0  
3  0.0      290000.0  
4  0.0      290000.0  

[5 rows x 260 columns]

In [280]:
Flat_building_pp.shape

(121934, 260)

In [281]:
Flat_building_pp.columns

Index([         'pc1',          'pc2',          'pc3',          'pc4',
                'pc5',          'pc6',           'p7',          'pc8',
                'pc9',         'pc10',
       ...
                  240,            241,            242,            243,
                  244,            245,            246,            247,
                  248, 'resale_price'],
      dtype='object', length=260)

# TRAIN TEST Split

In [288]:
y = Flat_building_pp["resale_price"]
x = Flat_building_pp.drop(["resale_price"], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 123)

In [289]:
y.shape

(121934,)

In [290]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(85353, 259)
(36581, 259)
(85353,)
(36581,)


# Model Building

# DecisionTree Regressor

In [291]:
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressorDT = DecisionTreeRegressor(random_state = 123)  
  
# fit the regressor with X and Y data 
regressorDT.fit(X_train, Y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=123, splitter='best')

In [292]:
Y_pred_DT = regressorDT.predict(X_test) 
   

In [293]:
from sklearn.metrics import mean_squared_error
from math import sqrt



In [294]:
rms = sqrt(mean_squared_error(Y_test, Y_pred_DT))

In [295]:
print(rms)

13299.009317474842


In [296]:
test_set_rmse_DT = (np.sqrt(mean_squared_error(Y_test, Y_pred_DT)))

#test_set_r2_DT = r2_score(Y_test, Y_pred_DT)

In [297]:
print(test_set_rmse_DT)

13299.009317474842


# Decision tree gridCV

In [338]:
from sklearn.tree import DecisionTreeRegressor

In [302]:

dtm = DecisionTreeRegressor()





In [339]:
param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [6,8,10],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

In [340]:

grid_cv_dtm = GridSearchCV(dtm, param_grid, cv=5)

In [342]:
grid_cv_dtm.fit(X_train, Y_train)

In [ ]:
Y_pred_DT_cv = grid_cv_dtm.predict(X_test) 

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt


In [ ]:
rms = sqrt(mean_squared_error(Y_test, Y_pred_DT_cv))

In [ ]:
print(rms)

In [ ]:
test_set_rmse_DT = (np.sqrt(mean_squared_error(Y_test, Y_pred_DT_cv)))

test_set_r2_DT = r2_score(Y_test, Y_pred_DT_cv)

In [ ]:
print("R-Squared::{}".format(grid_cv_dtm.best_score_))
print("Best Hyperparameters::\n{}".format(grid_cv_dtm.best_params_))

# Random Forest regression


In [307]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from math import sqrt

In [308]:
# Create a based model
regressorrfc = RandomForestRegressor()

param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20],
    'max_features': [2, 3,10],
    'min_samples_leaf': [3, 4,7],
    'min_samples_split': [8, 10,15],
    'n_estimators': [10,20,30,40]
}


# Instantiate the grid search model
grid_search_rfc = GridSearchCV(estimator = regressorrfc, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [309]:
grid_search_rfc.fit(X_train, Y_train) 

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   26.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:  3.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth':

In [310]:
grid_search_rfc.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 30}

In [311]:
Y_train_pred_rfc=grid_search_rfc.predict(X_train)

In [312]:
Y_pred_rfc = grid_search_rfc.predict(X_test)

In [313]:
rms = sqrt(mean_squared_error(Y_test, Y_pred_rfc))

In [314]:
print(rms)

40670.21372364201


In [ ]:
pip install rfpimp

# To validate the PCA components in Random Forest Important features

In [327]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances


In [334]:
def r2(grid_search_rfc, X_train, Y_train):
    return r2_score(Y_train, grid_search_rfc.predict(X_train))

perm_imp_rfpimp = permutation_importances(grid_search_rfc, X_train, Y_train, r2)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:    8.8s finished


In [337]:
print(perm_imp_rfpimp)

           Importance
Feature              
p7       1.116804e-01
pc4      1.015882e-01
23       8.691246e-02
pc1      6.144519e-02
25       6.063963e-02
pc2      5.319967e-02
26       4.937096e-02
pc8      2.971678e-02
pc3      1.798353e-02
11       1.444987e-02
pc6      1.440990e-02
pc5      1.356571e-02
pc10     1.338030e-02
pc9      8.962382e-03
24       8.927035e-03
22       8.446153e-03
227      8.165553e-03
2        7.632507e-03
4        6.178070e-03
6        6.150624e-03
198      6.137636e-03
241      5.305362e-03
225      5.305147e-03
63       5.253231e-03
17       5.157215e-03
3        3.644493e-03
16       3.572781e-03
7        3.556021e-03
248      3.399424e-03
12       2.762957e-03
...               ...
177      0.000000e+00
175      0.000000e+00
163      0.000000e+00
173      0.000000e+00
174      0.000000e+00
171      0.000000e+00
170      0.000000e+00
169      0.000000e+00
168      0.000000e+00
167      0.000000e+00
166      0.000000e+00
165      0.000000e+00
164      0

# SVM Regression

In [789]:
from sklearn.svm import SVR

In [791]:

regressorsv = SVR(kernel = 'rbf', gamma=0.5, C=1)


In [ ]:
regressorsv.fit(X_train, Y_train) 

In [ ]:
Y_pred_sv = regressorsv.predict(X_test)

In [ ]:
rms = sqrt(mean_squared_error(Y_test, Y_pred_sv))


# XG boost modell

In [135]:
import xgboost as xgb
from math import sqrt

In [129]:
xgb_clf = xgb.XGBRegressor() 

parameters = {'n_estimators': [12, 10, 20], 'max_depth':[3,5,7,9]}
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=parameters, cv=10, n_jobs=-1)
    
    
    
    

In [131]:
grid_search.fit(X_train, Y_train)

[16:54:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 9],
                         'n_estimators': [12, 10, 20]},
             pr

In [132]:
best_parameters=grid_search.best_estimator_.get_params()

In [133]:
Y_pred_xg = grid_search.predict(X_test)

In [136]:
rms = sqrt(mean_squared_error(Y_test, Y_pred_xg))

In [137]:
print(rms)

59656.83212243148


# Ensemble learning -  Stacking

In [ ]:
stack_train = pd.DataFrame([Y_train_pred_xg,Y_train_pred_sv,Y_train_pred_DT])
stack_test = pd.DataFrame([Y_pred_xg,Y_pred_sv,Y_pred_DT])

In [ ]:
stack_train = stack_train.T
stack_test = stack_test.T

In [ ]:
stack_train.columns = ['Y_pred_train_lrc','train_pred_rfc','y_pred_train_knn','train_pred_gbc']
stack_test.columns = ['Y_pred_test_lrc','test_pred_rfc','y_pred_test_knn','test_pred_gbc']

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressorDT = DecisionTreeRegressor(random_state = 0)  
  
# fit the regressor with X and Y data 
stackregressorDT.fit(stack_train, stack_test) 

In [ ]:
Y_stack_pred_DT = stackregressorDT.predict(stack_test) 

In [ ]:
rms = sqrt(mean_squared_error(stack_test, Y_stack_pred_DT))

In [ ]:
test_set_rmse_DT = (np.sqrt(mean_squared_error(stack_test, Y_stack_pred_DT)))

test_set_r2_DT = r2_score(Y_stack_pred_DT, Y_stack_pred_DT)

# Summary

# Out of all the models, Decision tree gave less RMSE